# **Urban Productions: Movie Production Analysis**. 

## **Business Understanding**

### **Overview**
Urban Enterprises is a company that has decided to venture into the movie production industry and want to build a new studio.   
The head of studio needs to know what type of films/movies to create.  
Our goal is to analyze movie performance data to provide insights on movie production.  
We are using datasets from: *Box Office Mojo*, *IMDB*, *The Numbers*

### **Problem Statement**
The company barely has any knowledge on what it entails to run a successful movie production business. Therefore, it requires research done on what types of films are currently doing the best in the industry. The stakeholders need meaningful insights on movie trends ie the top performing genres, in order to make decisions that will eventually be profitable for the investment.


### **Objectives**

1. Determine the best movie genres by examining total gross and movie ratings for the business to take steps that will maximize their profits and ensure return of investment.

2. Evaluate if there is correlation between movies total gross and their ratings

3. Compare production budgets for movies with thir total gross. Examine whether there is increase in production budget over the years, and if the gross is keping up with it.

4. Explore whether movie release dates/months affect their performance to suggest the best release strategies

5. Identify some of the most influential names in the industry that are associated with top performing movies

6. Verify if the movie studios have an effect on the total gross








## **Data Understanding**
